In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score,precision_recall_curve
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Standard rating
standard_rating = ['Seat_Comfort',
'Arrival_Time_Convenient',
'Catering',
'Onboard_Wifi_Service',
'Onboard_Entertainment',
'Online_Support',
'Ease_of_Online_Booking',
'Onboard_Service',
'Legroom',
'Baggage_Handling',
'CheckIn_Service',
'Cleanliness',
'Online_Boarding']
targets = ['Customer_Type','Type_Travel']

### 0/ Import for training

**Category to numeric**

In [3]:
df = pd.read_csv("1.df correlation filled.csv")
expected_df_shape = df.copy().shape
expected_df_shape

(94379, 25)

In [4]:
df.drop(columns=['ID','Overall_Experience'],inplace=True)

In [5]:
for col in standard_rating:
    df[col].replace(['Excellent', 'Good', 'Acceptable', 'Needs Improvement', 'Poor', 'Extremely Poor'], [5, 4, 3, 2, 1, 0], inplace = True)

In [6]:
df['Platform_Location'].replace(['Very Convenient', 'Convenient', 'Manageable', 'Needs Improvement', 'Inconvenient','Very Inconvenient'], 
                                [5,4,3,2,1,0], inplace = True)

In [7]:
# Creating list of dummy columns
get_dummies = ['Seat_Class', 'Gender', 'Travel_Class']

# Creating dummy variables
df = pd.get_dummies(data = df, columns = get_dummies, drop_first = True)  

Target encoding

In [8]:
df['Type_Travel'].replace(['Business Travel', 'Personal Travel'], 
                                [1,0], inplace = True)
df['Customer_Type'].replace(['Loyal Customer', 'Disloyal Customer'], 
                                [1,0], inplace = True)

**Check**

In [9]:
df.head(20)

,Seat_Comfort,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Customer_Type,Age,Type_Travel,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Seat_Class_Ordinary,Gender_Male,Travel_Class_Eco
0,2,5,5,5,4,2,3,2,2,3,2,4,2,1,1.0,52.0,NaN,272,0.0,5.0,0,0,0
1,1,5,1,2,4,1,4,4,5,2,1,2,4,4,1.0,48.0,0.0,2200,9.0,0.0,1,1,1
2,2,2,2,2,2,4,5,5,5,5,5,4,5,5,1.0,43.0,1.0,1061,77.0,119.0,0,0,0
3,3,2,3,2,3,2,3,3,3,3,3,4,3,3,1.0,44.0,1.0,780,13.0,18.0,1,0,0
4,3,3,3,3,2,4,5,4,4,4,4,4,4,4,1.0,50.0,1.0,1981,0.0,0.0,1,0,0
5,2,2,3,2,4,5,4,3,3,4,3,3,3,3,1.0,44.0,1.0,2810,0.0,0.0,1,1,0
6,4,5,4,1,4,4,4,4,4,5,2,5,3,4,1.0,56.0,0.0,2029,0.0,0.0,0,1,1
7,3,1,3,1,5,5,4,4,4,3,4,4,4,5,1.0,65.0,0.0,853,0.0,3.0,0,1,0
8,2,2,2,2,3,2,3,3,3,3,2,3,2,3,1.0,22.0,0.0,1636,1.0,0.0,0,1,1
9,2,4,4,4,3,4,4,2,2,2,2,3,2,3,1.0,57.0,1.0,306,0.0,0.0,1,1,0


**Check datatype (should be almost all int numeric, except age, minutes and 2 targets)**

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94379 entries, 0 to 94378
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Seat_Comfort             94379 non-null  int64  
 1   Arrival_Time_Convenient  94379 non-null  int64  
 2   Catering                 94379 non-null  int64  
 3   Platform_Location        94379 non-null  int64  
 4   Onboard_Wifi_Service     94379 non-null  int64  
 5   Onboard_Entertainment    94379 non-null  int64  
 6   Online_Support           94379 non-null  int64  
 7   Ease_of_Online_Booking   94379 non-null  int64  
 8   Onboard_Service          94379 non-null  int64  
 9   Legroom                  94379 non-null  int64  
 10  Baggage_Handling         94379 non-null  int64  
 11  CheckIn_Service          94379 non-null  int64  
 12  Cleanliness              94379 non-null  int64  
 13  Online_Boarding          94379 non-null  int64  
 14  Customer_Type         

**Check range of each**

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Seat_Comfort,94379.0,2.839233,1.392574,0.0,2.0,3.0,4.0,5.0
Arrival_Time_Convenient,94379.0,2.993833,1.507405,0.0,2.0,3.0,4.0,5.0
Catering,94379.0,2.850157,1.439156,0.0,2.0,3.0,4.0,5.0
Platform_Location,94379.0,2.991068,1.308160,0.0,2.0,3.0,4.0,5.0
Onboard_Wifi_Service,94379.0,3.248297,1.319507,0.0,2.0,3.0,4.0,5.0
Onboard_Entertainment,94379.0,3.382394,1.346257,0.0,2.0,4.0,4.0,5.0
Online_Support,94379.0,3.519374,1.308160,0.0,3.0,4.0,5.0,5.0
Ease_of_Online_Booking,94379.0,3.470316,1.305509,0.0,2.0,4.0,5.0,5.0
Onboard_Service,94379.0,3.488456,1.262326,0.0,3.0,4.0,4.0,5.0
Legroom,94379.0,3.483105,1.292205,0.0,2.0,4.0,5.0,5.0


Nothing out of range.

### Prep data

In [12]:
X = df.drop(columns = targets)

In [13]:
X = X.values.astype(float)
Y1 = df['Customer_Type']
Y2 = df['Type_Travel']

X1 = X[Y1.notnull()]
Y1 = Y1[Y1.notnull()]

X2 = X[Y2.notnull()]
Y2 = Y2[Y2.notnull()]

Check if X1,Y1 has same rows and X2,Y2 has same rows

In [14]:
X1.shape[0]==Y1.shape[0], X2.shape[0]==Y2.shape[0]

(True, True)

### 1a/ Customer model

In [15]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, Y1, test_size=0.2, stratify=Y1, random_state=14)

# scale here for 2.1, 2.2
scaler1 = preprocessing.StandardScaler()
X1_train = scaler1.fit_transform(X1_train)
X1_test = scaler1.transform(X1_test)

X1_train.shape, y1_train.shape, X1_test.shape, y1_test.shape,

((68342, 21), (68342,), (17086, 21), (17086,))

**Loop**

In [16]:
for k in np.arange(3,16,1):
    KNN_customer = KNeighborsClassifier(n_neighbors = k).fit(X1_train,y1_train)
    print('k = ',k)
    print("Train set Accuracy: ", accuracy_score(y1_train, KNN_customer.predict(X1_train)))
    print("Test set Accuracy: ", accuracy_score(y1_test, KNN_customer.predict(X1_test)))
    print('-'*50)
    del KNN_customer

k =  3
Train set Accuracy:  0.9473383863509994
Test set Accuracy:  0.9034882359826759
--------------------------------------------------
k =  4
Train set Accuracy:  0.9365543882239326
Test set Accuracy:  0.8979281282921691
--------------------------------------------------
k =  5
Train set Accuracy:  0.9355008633051418
Test set Accuracy:  0.9047173124195248
--------------------------------------------------
k =  6
Train set Accuracy:  0.9309502209475872
Test set Accuracy:  0.902259159545827
--------------------------------------------------
k =  7
Train set Accuracy:  0.928448099265459
Test set Accuracy:  0.9054781692613836
--------------------------------------------------
k =  8
Train set Accuracy:  0.9262825202657224
Test set Accuracy:  0.9042490928245347
--------------------------------------------------
k =  9
Train set Accuracy:  0.9240145152322145
Test set Accuracy:  0.9068828280463538
--------------------------------------------------
k =  10
Train set Accuracy:  0.923239003833

**Model for imputing**

In [17]:
k_customer = 11
KNN_customer = KNeighborsClassifier(n_neighbors = k_customer).fit(X1_train,y1_train)
print('k = ',k_customer)
print("Train set Accuracy: ", accuracy_score(y1_train, KNN_customer.predict(X1_train)))
print("Test set Accuracy: ", accuracy_score(y1_test, KNN_customer.predict(X1_test)))

k =  11
Train set Accuracy:  0.9208685727663809
Test set Accuracy:  0.9091068711225565


In [18]:
Y_customer_predict = KNN_customer.predict(scaler1.transform(X))
np.unique(Y_customer_predict, return_counts=True)

(array([0., 1.]), array([14211, 80168], dtype=int64))

### 2/ Type_Travel model

In [19]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.2, stratify=Y2, random_state=14)

# scale here for 2.1, 2.2
scaler2 = preprocessing.StandardScaler()
X2_train = scaler2.fit_transform(X2_train)
X2_test = scaler2.transform(X2_test)

X2_train.shape, y2_train.shape, X2_test.shape, y2_test.shape

((68122, 21), (68122,), (17031, 21), (17031,))

**Loop**

In [20]:
# make sure the loop is X2, y2, not 1
for k in np.arange(3,16,1):
    KNN_Type_Travel = KNeighborsClassifier(n_neighbors = k).fit(X2_train,y2_train)
    print('k = ',k)
    print("Train set Accuracy: ", accuracy_score(y2_train, KNN_Type_Travel.predict(X2_train)))
    print("Test set Accuracy: ", accuracy_score(y2_test, KNN_Type_Travel.predict(X2_test)))
    print('-'*50)
    del KNN_Type_Travel

k =  3
Train set Accuracy:  0.9323566542379847
Test set Accuracy:  0.8764605718983031
--------------------------------------------------
k =  4
Train set Accuracy:  0.9238571973811691
Test set Accuracy:  0.8778110504374376
--------------------------------------------------
k =  5
Train set Accuracy:  0.9183083291741287
Test set Accuracy:  0.8818037695966179
--------------------------------------------------
k =  6
Train set Accuracy:  0.9140219018819178
Test set Accuracy:  0.8825670835535201
--------------------------------------------------
k =  7
Train set Accuracy:  0.9099263086814833
Test set Accuracy:  0.8814514708472785
--------------------------------------------------
k =  8
Train set Accuracy:  0.9088253427673879
Test set Accuracy:  0.8846221595913335
--------------------------------------------------
k =  9
Train set Accuracy:  0.9054784063885382
Test set Accuracy:  0.8853267570900123
--------------------------------------------------
k =  10
Train set Accuracy:  0.9053022518

**Model for imputing**

In [21]:
k_travel = 9
KNN_travel = KNeighborsClassifier(n_neighbors = k_travel).fit(X2_train,y2_train)
print('k = ',k_travel)
print("Train set Accuracy: ", accuracy_score(y2_train, KNN_travel.predict(X2_train)))
print("Test set Accuracy: ", accuracy_score(y2_test, KNN_travel.predict(X2_test)))

k =  9
Train set Accuracy:  0.9054784063885382
Test set Accuracy:  0.8853267570900123


In [22]:
Y_travel_predict = KNN_travel.predict(scaler2.transform(X))
np.unique(Y_travel_predict, return_counts=True)

(array([0., 1.]), array([26363, 68016], dtype=int64))

### 3/ Impute (train) model

- Re import the main df, just for the purpose of concat/replace nan from what we had predicted above
- No transform or whatsoever. Just concat/replace
- That's why don't drop any targets column. All data must be kept intact.

In [23]:
df = pd.read_csv("1.df correlation filled.csv")

Check target 1 before replace na

In [24]:
df['Customer_Type'].value_counts()

Loyal Customer       69823
Disloyal Customer    15605
Name: Customer_Type, dtype: int64

Replace null and convert back to original display

In [25]:
df['Customer_Type'].fillna(pd.Series(Y_customer_predict), inplace= True)
df['Customer_Type'].replace([1,0],
                           ['Loyal Customer', 'Disloyal Customer'],
                           inplace = True
                          )
df['Customer_Type'].value_counts()

Loyal Customer       77391
Disloyal Customer    16988
Name: Customer_Type, dtype: int64

Check target 2 before replacing

In [26]:
df['Type_Travel'].value_counts()

Business Travel    58617
Personal Travel    26536
Name: Type_Travel, dtype: int64

In [27]:
df['Type_Travel'].fillna(pd.Series(Y_travel_predict), inplace= True)
df['Type_Travel'].replace([1,0],
                           ['Business Travel', 'Personal Travel'],
                           inplace = True
                          )
df['Type_Travel'].value_counts()

Business Travel    65334
Personal Travel    29045
Name: Type_Travel, dtype: int64

In [28]:
# Check if same expected shape
df.shape, expected_df_shape

((94379, 25), (94379, 25))

In [29]:
df.to_csv('2.0 df_KNN_imputed.csv', index=False)

### 4/ Impute (test) set

Impute test set:
- First we have to import, transform or not thereof, to make a prediction
- Be careful at preprocess - fit - transform stuff
- use same scaler

- Here we don't need to split to different train/test.
- Use the model above to make prediction

- Only after that, we import again to replace null and export.

In [30]:
df_submit = pd.read_csv('1.df_submit correlation filled.csv')
expected_submit_shape = df_submit.copy().shape
expected_submit_columns = df_submit.copy().columns
expected_submit_shape

(35602, 24)

- Expected shape here is 1 less column than the train set, because no target columns.

Preprocess to make prediction

In [31]:
# Basic rating transform and dummy, use same column list as above.
for col in standard_rating:
    df_submit[col].replace(['Excellent', 'Good', 'Acceptable', 'Needs Improvement', 'Poor', 'Extremely Poor'], [5, 4, 3, 2, 1, 0], inplace = True)    
df_submit['Platform_Location'].replace(['Very Convenient', 'Convenient', 'Manageable', 'Needs Improvement', 'Inconvenient','Very Inconvenient'], 
                                [5,4,3,2,1,0], inplace = True) 
df_submit = pd.get_dummies(data = df_submit, columns = get_dummies, drop_first = True)  

# Make the X_testset.
X_testset = df_submit.drop(columns = ['ID'] + targets).values.astype(float)

# Make sure no fit, only transform
# Make sure it's X_testset and not X
X_test_customer = scaler1.transform(X_testset)
X_test_travel = scaler2.transform(X_testset)
X_test_customer.shape,X_test_travel.shape  #Have 3 less column because ID, 2 target columns

((35602, 21), (35602, 21))

Make prediction on their own X_test_<customer/travel>

In [32]:
Y_customer_predict_testset = KNN_customer.predict(X_test_customer)
Y_travel_predict_testset = KNN_travel.predict(X_test_travel)

Quality check

In [33]:
np.unique(Y_customer_predict_testset, return_counts=True)

(array([0., 1.]), array([ 5322, 30280], dtype=int64))

In [34]:
np.unique(Y_travel_predict_testset, return_counts=True)

(array([0., 1.]), array([ 9635, 25967], dtype=int64))

**Import again**

In [35]:
df_submit = pd.read_csv('1.df_submit correlation filled.csv')

In [36]:
df_submit['Customer_Type'].fillna(pd.Series(Y_customer_predict_testset), inplace= True)
df_submit['Customer_Type'].replace([1,0],
                           ['Loyal Customer', 'Disloyal Customer'],
                           inplace = True
                          )
df_submit['Customer_Type'].value_counts()

Loyal Customer       29231
Disloyal Customer     6371
Name: Customer_Type, dtype: int64

In [37]:
df_submit['Type_Travel'].fillna(pd.Series(Y_travel_predict_testset), inplace= True)
df_submit['Type_Travel'].replace([1,0],
                           ['Business Travel', 'Personal Travel'],
                           inplace = True
                          )
df_submit['Type_Travel'].value_counts()

Business Travel    24838
Personal Travel    10764
Name: Type_Travel, dtype: int64

In [38]:
df_submit.shape, expected_submit_shape # same size, looks good

((35602, 24), (35602, 24))

In [39]:
df_submit.columns == expected_submit_columns # if all true, good, false -> forget to re im

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [40]:
df_submit.to_csv('2.0 df_submit_KNN_imputed.csv', index=False)

### Check

Check if all False

In [41]:
df.isnull().any()

ID                         False
Overall_Experience         False
Seat_Comfort               False
Seat_Class                 False
Arrival_Time_Convenient    False
Catering                   False
Platform_Location          False
Onboard_Wifi_Service       False
Onboard_Entertainment      False
Online_Support             False
Ease_of_Online_Booking     False
Onboard_Service            False
Legroom                    False
Baggage_Handling           False
CheckIn_Service            False
Cleanliness                False
Online_Boarding            False
Gender                     False
Customer_Type              False
Age                        False
Type_Travel                False
Travel_Class               False
Travel_Distance            False
Departure_Delay_in_Mins    False
Arrival_Delay_in_Mins      False
dtype: bool

In [42]:
df_submit.isnull().any()

ID                         False
Seat_Comfort               False
Seat_Class                 False
Arrival_Time_Convenient    False
Catering                   False
Platform_Location          False
Onboard_Wifi_Service       False
Onboard_Entertainment      False
Online_Support             False
Ease_of_Online_Booking     False
Onboard_Service            False
Legroom                    False
Baggage_Handling           False
CheckIn_Service            False
Cleanliness                False
Online_Boarding            False
Gender                     False
Customer_Type              False
Age                        False
Type_Travel                False
Travel_Class               False
Travel_Distance            False
Departure_Delay_in_Mins    False
Arrival_Delay_in_Mins      False
dtype: bool

Check if same column in one place

Check if any weird values

In [43]:
df.head(20)

,ID,Overall_Experience,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
0,98800001,0,Needs Improvement,Green Car,Excellent,Excellent,Very Convenient,Good,Needs Improvement,Acceptable,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Good,Needs Improvement,Poor,Female,Loyal Customer,52.0,Business Travel,Business,272,0.0,5.0
1,98800002,0,Poor,Ordinary,Excellent,Poor,Needs Improvement,Good,Poor,Good,Good,Excellent,Needs Improvement,Poor,Needs Improvement,Good,Good,Male,Loyal Customer,48.0,Personal Travel,Eco,2200,9.0,0.0
2,98800003,1,Needs Improvement,Green Car,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Good,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,Female,Loyal Customer,43.0,Business Travel,Business,1061,77.0,119.0
3,98800004,0,Acceptable,Ordinary,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Acceptable,Female,Loyal Customer,44.0,Business Travel,Business,780,13.0,18.0
4,98800005,1,Acceptable,Ordinary,Acceptable,Acceptable,Manageable,Needs Improvement,Good,Excellent,Good,Good,Good,Good,Good,Good,Good,Female,Loyal Customer,50.0,Business Travel,Business,1981,0.0,0.0
5,98800006,1,Needs Improvement,Ordinary,Needs Improvement,Acceptable,Needs Improvement,Good,Excellent,Good,Acceptable,Acceptable,Good,Acceptable,Acceptable,Acceptable,Acceptable,Male,Loyal Customer,44.0,Business Travel,Business,2810,0.0,0.0
6,98800007,0,Good,Green Car,Excellent,Good,Inconvenient,Good,Good,Good,Good,Good,Excellent,Needs Improvement,Excellent,Acceptable,Good,Male,Loyal Customer,56.0,Personal Travel,Eco,2029,0.0,0.0
7,98800008,0,Acceptable,Green Car,Poor,Acceptable,Inconvenient,Excellent,Excellent,Good,Good,Good,Acceptable,Good,Good,Good,Excellent,Male,Loyal Customer,65.0,Personal Travel,Business,853,0.0,3.0
8,98800009,0,Needs Improvement,Green Car,Needs Improvement,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Male,Loyal Customer,22.0,Personal Travel,Eco,1636,1.0,0.0
9,98800010,0,Needs Improvement,Ordinary,Good,Good,Convenient,Acceptable,Good,Good,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Acceptable,Male,Loyal Customer,57.0,Business Travel,Business,306,0.0,0.0


In [44]:
df_submit.head(10)

,ID,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
0,99900001,Acceptable,Green Car,Acceptable,Acceptable,Manageable,Needs Improvement,Excellent,Good,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Poor,Female,Loyal Customer,36.0,Business Travel,Business,532,0.0,0.0
1,99900002,Extremely Poor,Ordinary,Good,Poor,Manageable,Acceptable,Poor,Acceptable,Acceptable,Excellent,Acceptable,Good,Acceptable,Excellent,Acceptable,Female,Disloyal Customer,21.0,Business Travel,Business,1425,9.0,28.0
2,99900003,Excellent,Ordinary,Excellent,Excellent,Very Convenient,Excellent,Excellent,Excellent,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Good,Needs Improvement,Excellent,Male,Loyal Customer,60.0,Business Travel,Business,2832,0.0,0.0
3,99900004,Acceptable,Green Car,Excellent,Acceptable,Very Convenient,Poor,Acceptable,Excellent,Poor,Acceptable,Needs Improvement,Excellent,Excellent,Excellent,Poor,Female,Loyal Customer,29.0,Personal Travel,Eco,1352,0.0,0.0
4,99900005,Excellent,Ordinary,Extremely Poor,Excellent,Needs Improvement,Excellent,Excellent,Excellent,Excellent,Excellent,Acceptable,Excellent,Excellent,Excellent,Excellent,Male,Disloyal Customer,18.0,Business Travel,Business,1610,17.0,0.0
5,99900006,Poor,Green Car,Poor,Poor,Inconvenient,Good,Good,Good,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Male,Loyal Customer,49.0,Business Travel,Business,382,89.0,72.0
6,99900007,Acceptable,Ordinary,Acceptable,Acceptable,Inconvenient,Excellent,Acceptable,Excellent,Excellent,Acceptable,Acceptable,Good,Acceptable,Excellent,Excellent,Male,Disloyal Customer,40.0,Business Travel,Business,1761,0.0,0.0
7,99900008,Needs Improvement,Ordinary,Needs Improvement,Needs Improvement,Needs Improvement,Excellent,Poor,Excellent,Good,Good,Good,Good,Excellent,Good,Excellent,Female,Loyal Customer,11.0,Personal Travel,Eco,3989,0.0,9.0
8,99900009,Acceptable,Ordinary,Good,Good,Convenient,Needs Improvement,Acceptable,Good,Good,Good,Good,Acceptable,Needs Improvement,Good,Acceptable,Male,Loyal Customer,57.0,Business Travel,Business,2731,0.0,0.0
9,99900010,Needs Improvement,Green Car,Excellent,Excellent,Very Convenient,Needs Improvement,Needs Improvement,Needs Improvement,Acceptable,Good,Good,Acceptable,Needs Improvement,Acceptable,Needs Improvement,Female,Loyal Customer,43.0,Business Travel,Eco,2645,222.0,223.0
